
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [1]:
 !pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

Cloning into 'tf-levenberg-marquardt'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 53 (delta 31), reused 29 (delta 14), pack-reused 0
Receiving objects: 100% (53/53), 32.99 KiB | 6.60 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y4", label="solidos")

In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-02-27 00:18:26.319749: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-27 00:18:26.354579: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-27 00:18:26.354612: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-27 00:18:26.355903: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-27 00:18:26.362157: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-27 00:18:26.363226: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [9]:
tester.Loop(sort_by='r2',
            boundarie=0.5,
            hidden_sizes = [[15, 7], [30, 15], [30, 25]],
            regularizers=[0.02, 0.1],
            learning_rate=[0.02])

Testando combinacao1: Hidden Size=[15, 7], regularizer=0.02, learning_rate=0.02
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  2.0831269780001094
Stopped at epoch:  28
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.20785295099994983
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.4011213180001505
Stopped at epoch:  0
1/1 [==============================] - 0s 18ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.03164409799956047
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.026939064000544022
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.026777215999572945
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 7 +++

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_3_0_0,0.504723,-0.084434,0.312703,0.084460,59.730537,157.314896,82.997787,122.342140
model_3_0_2,0.515321,-0.156228,0.176409,-0.014764,58.452473,167.729843,99.456604,135.601303
model_3_0_1,0.516906,-0.099921,0.297839,0.069237,58.261250,159.561600,84.792831,124.376251
model_3_0_4,0.520231,-0.149482,0.110082,-0.039095,57.860298,166.751282,107.466339,138.852509
model_3_0_3,0.522615,-0.147458,0.152649,-0.019828,57.572758,166.457687,102.325851,136.277969
model_3_0_5,0.524507,-0.141158,0.105964,-0.036061,57.344578,165.543686,107.963615,138.447174
model_3_0_6,0.533073,-0.164572,0.036880,-0.078897,56.311489,168.940308,116.306190,144.171280
model_3_0_8,0.540236,-0.405370,0.190465,-0.151975,55.447685,203.871994,97.759201,153.936523
model_3_0_7,0.551339,-0.277137,0.168099,-0.087788,54.108681,185.269592,100.460182,145.359299
model_3_0_9,0.557419,-0.301246,0.138807,-0.114101,53.375435,188.767151,103.997444,148.875549


DataFrame salvo em ../results/metrics_3_0
+++++++++++ [3_1] | 1 ++++++++++++++++++
Elapsed time:  0.2347108379999554
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [3_1] | 2 ++++++++++++++++++
Elapsed time:  0.37600488899988704
Stopped at epoch:  10
1/1 [==============================] - 0s 14ms/step
+++++++++++ [3_1] | 3 ++++++++++++++++++
Elapsed time:  0.30961849100003747
Stopped at epoch:  9
1/1 [==============================] - 0s 14ms/step
+++++++++++ [3_1] | 4 ++++++++++++++++++
Elapsed time:  0.21864301100049488
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [3_1] | 5 ++++++++++++++++++
Elapsed time:  0.2184942530002445
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [3_1] | 6 ++++++++++++++++++
Elapsed time:  0.2243609209999704
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [3_1] | 7 ++++++++++++++++++
Elapsed time:  0.22743631

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_3_1_1,0.500369,0.022661,-0.166982,-0.047852,60.255676,123.555244,138.537933,130.606125
model_3_1_3,0.509972,0.097435,-0.296201,-0.065624,59.097557,114.102364,153.878113,132.821335
model_3_1_2,0.511351,0.109499,-0.293616,-0.057996,58.931175,112.577179,153.571213,131.870483
model_3_1_4,0.529542,0.024048,-0.097178,-0.015820,56.737316,123.379951,130.251205,126.613586
model_3_1_5,0.533550,0.046070,-0.102210,-0.006249,56.253998,120.595963,130.848557,125.420654
model_3_1_0,0.553171,0.485305,0.717145,0.596848,53.887646,65.067757,33.579056,50.249546


DataFrame salvo em ../results/metrics_3_1
+++++++++++ [3_2] | 1 ++++++++++++++++++
Elapsed time:  0.22014152399970044
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [3_2] | 2 ++++++++++++++++++
Elapsed time:  0.2547468789998675
Stopped at epoch:  7
1/1 [==============================] - 0s 14ms/step
+++++++++++ [3_2] | 3 ++++++++++++++++++
Elapsed time:  0.2303461769997739
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [3_2] | 4 ++++++++++++++++++
Elapsed time:  0.26443514800030243
Stopped at epoch:  8
1/1 [==============================] - 0s 13ms/step
+++++++++++ [3_2] | 5 ++++++++++++++++++
Elapsed time:  0.44368341000063083
Stopped at epoch:  12
1/1 [==============================] - 0s 14ms/step
+++++++++++ [3_2] | 6 ++++++++++++++++++
Elapsed time:  0.47950187500009633
Stopped at epoch:  14
1/1 [==============================] - 0s 14ms/step
+++++++++++ [3_2] | 7 ++++++++++++++++++
Elapsed time:  0.231371

...

DataFrame salvo em ../results/metrics_6_7
+++++++++++ [6_8] | 1 ++++++++++++++++++
Elapsed time:  0.1772765450004954
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [6_8] | 2 ++++++++++++++++++
Elapsed time:  0.19387541499963845
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [6_8] | 3 ++++++++++++++++++
Elapsed time:  0.1621621310005139
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [6_8] | 4 ++++++++++++++++++
Elapsed time:  0.16080044499994983
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [6_8] | 5 ++++++++++++++++++
Elapsed time:  0.15755129099943588
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [6_8] | 6 ++++++++++++++++++
Elapsed time:  0.16047173399965686
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [6_8] | 7 ++++++++++++++++++
Elapsed time:  0.16395370

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_6_8_23,0.837785,-0.078562,0.765755,0.434371,19.563234,88.190590,31.652370,61.584316
model_6_8_24,0.839904,-0.064005,0.766065,0.440339,19.307648,87.000282,31.610477,60.934498
model_6_8_19,0.841240,0.036778,0.752486,0.472477,19.146515,78.759575,33.445301,57.435436
model_6_8_18,0.842410,0.058976,0.750647,0.480229,19.005402,76.944572,33.693787,56.591446
model_6_8_22,0.843052,0.012647,0.754064,0.463806,18.928022,80.732704,33.232071,58.379498
model_6_8_21,0.845283,0.027456,0.759031,0.472596,18.658878,79.521858,32.560913,57.422424
model_6_8_20,0.846058,0.036893,0.759314,0.476511,18.565475,78.750221,32.522743,56.996185
model_6_8_14,0.877067,0.354990,0.859292,0.661374,14.825709,52.740406,19.013182,36.868797
model_6_8_17,0.879215,0.349095,0.854182,0.656046,14.566765,53.222420,19.703697,37.448856
model_6_8_13,0.879936,0.371555,0.872629,0.675749,14.479743,51.385956,17.211033,35.303665


DataFrame salvo em ../results/metrics_6_8
+++++++++++ [6_9] | 1 ++++++++++++++++++
Elapsed time:  0.235876468999777
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [6_9] | 2 ++++++++++++++++++
Elapsed time:  0.29585486500036495
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [6_9] | 3 ++++++++++++++++++
Elapsed time:  0.19962308000049234
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [6_9] | 4 ++++++++++++++++++
Elapsed time:  0.23092335300043487
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [6_9] | 5 ++++++++++++++++++
Elapsed time:  0.30498424699999305
Stopped at epoch:  9
1/1 [==============================] - 0s 15ms/step
+++++++++++ [6_9] | 6 ++++++++++++++++++
Elapsed time:  0.18205411999952048
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [6_9] | 7 ++++++++++++++++++
Elapsed time:  0.18302030

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_6_9_24,0.777819,0.089023,0.060074,0.200182,26.795141,97.621780,59.778099,79.812950
model_6_9_23,0.784356,0.131213,0.077790,0.229481,26.006687,93.100616,58.651360,76.889267
model_6_9_22,0.785822,0.145085,0.082696,0.238839,25.829878,91.614014,58.339394,75.955406
model_6_9_21,0.788402,0.163117,0.095257,0.252858,25.518820,89.681747,57.540489,74.556442
model_6_9_20,0.791767,0.189463,0.102789,0.270096,25.112967,86.858429,57.061508,72.836296
model_6_9_19,0.796841,0.220636,0.124337,0.294280,24.500986,83.517914,55.691032,70.422981
model_6_9_18,0.801393,0.249707,0.143827,0.316654,23.952021,80.402542,54.451534,68.190308
model_6_9_17,0.805029,0.280605,0.148173,0.335524,23.513615,77.091476,54.175148,66.307297
model_6_9_16,0.810871,0.317080,0.171645,0.363302,22.809025,73.182762,52.682358,63.535408
model_6_9_15,0.814152,0.338460,0.190470,0.381101,22.413311,70.891724,51.485107,61.759220


DataFrame salvo em ../results/metrics_6_9
